In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
print(tf.__version__)
import tensorflow.keras.layers as tfl
from tensorflow import keras
from datetime import datetime

print("All packages imported!")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/digit-recognizer'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2.4.1
All packages imported!
/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [28]:
train_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test_data = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

print(train_data.head())
print(test_data.head())

   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  
0         0         0         

In [29]:
Y_train = np.array(train_data.label)
Y_train = Y_train.reshape(Y_train.shape[0],1)
Y_train = tf.one_hot(Y_train,depth = 10)
Y_train = np.array(Y_train)
print(Y_train.shape)
Y_train = Y_train.reshape(Y_train.shape[0],10)
print(Y_train.shape)
#print(Y_train)

X_train = train_data
X_train.drop(['label'],axis=1,inplace=True)
X_train = np.array(X_train)
print(X_train.shape)

X_train = np.reshape(X_train, (X_train.shape[0],-1, 28,1))
print(X_train.shape)

X_test = test_data
X_test = np.array(X_test)
print(X_test.shape)

X_test = np.reshape(X_test, (X_test.shape[0],-1, 28,1))

print(X_test.shape)

(42000, 1, 10)
(42000, 10)
(42000, 784)
(42000, 28, 28, 1)
(28000, 784)
(28000, 28, 28, 1)


In [30]:
ind = []
for i in range(X_test.shape[0]):
    ind.append(i+1)

test_data['ImageId'] = ind
test_data.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,ImageId
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [31]:
def convnet(inp_shape):
    #hyperparameters
    stride1 =1
    maxpool_stride1 = 2
    maxpool_size1 = 8
    no_f1 = 32
    f1 = 5

    stride2 =1
    maxpool_stride2 = 2
    maxpool_size2 = 8
    no_f2 = 16
    f2 = 2

    #Model definition

    input_img = tf.keras.Input(shape = inp_shape)

    Z1 = tfl.Conv2D(no_f1, (f1, f1), stride1, padding='same')(input_img)
    A1 = tfl.ReLU()(Z1)
    P1 = tfl.MaxPooling2D(pool_size=maxpool_size1, strides=maxpool_stride1, padding='same')(A1)
    Z2 = tfl.Conv2D(no_f2, (f2, f2), stride2, padding='same')(P1)
    A2 = tfl.ReLU()(Z2)
    P2 = tfl.MaxPooling2D(pool_size=maxpool_size2, strides=maxpool_stride2, padding='same')(A2)
    F = tfl.Flatten()(P2)
    outputs = tfl.Dense(10, activation="softmax")(F)

    model = tf.keras.Model(inputs=input_img, outputs=outputs)
    model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

    return model


In [34]:
conv_model = convnet((28,28,1))
conv_model.summary()

now1 = datetime.now().time() # time object1
conv_model.fit(X_train,Y_train,batch_size=64,epochs=5)
now2 = datetime.now().time() # time object2

print("Training Start Time =", now1)
print("Training End Time =", now2)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
re_lu_2 (ReLU)               (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 16)        2064      
_________________________________________________________________
re_lu_3 (ReLU)               (None, 14, 14, 16)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 16)          0   

In [35]:
X_test = X_test.astype("float32")

now1 = datetime.now().time() # time object1
predictions = conv_model.predict(X_test)
now2 = datetime.now().time() # time object2

no_predictions = np.argmax(predictions, axis=1)
print(predictions.shape)
print(no_predictions)

print("Prediction Start Time =", now1)
print("Prediction End Time =", now2)

(28000, 10)
[2 0 9 ... 3 9 2]
Prediction Start Time = 09:06:21.846712
Prediction End Time = 09:06:28.315061


In [36]:
output = pd.DataFrame({'ImageId': test_data.ImageId, 'Label': no_predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
